In [55]:
import numpy as np
import h5py
import cv2
import matplotlib.pyplot as plt
from scipy.io import loadmat
import sklearn
from skimage import exposure, morphology, io
from skimage.segmentation import active_contour
import os
import sys
import glob
from sklearn.cluster import KMeans

In [58]:
def check_mat_file_structure(file_path):
    # Load the .mat file using hdf5storage
    data = hdf5storage.loadmat(file_path)
    print("Keys in the .mat file:")
    for key in data.keys():
        print(key)
    return data

# Specify the path to one of your .mat files
file_path = '/Users/elizabethnemeti/Desktop/subset/1.mat'  # Adjust the path and filename as necessary

# Check the structure of the .mat file
mat_data = check_mat_file_structure(file_path)

Keys in the .mat file:
__header__
__version__
__globals__
image


In [72]:
def load_and_preprocess_data(data_dir, image_dimension=512):
    images = []
    files = os.listdir(data_dir)

    print(f"Total files found: {len(files)}")
    processed_count = 0

    for i, file in enumerate(files, start=1):
        try:
            mat_file = hdf5storage.loadmat(os.path.join(data_dir, file))  # Load the .mat file
            if 'image' in mat_file:
                image = mat_file['image']
                # Check if the image is 3D and handle accordingly:
                if image.ndim == 3:
                    # Select the middle slice from the volume
                    middle_slice_index = image.shape[2] // 2
                    image = image[:, :, middle_slice_index]
                image = cv2.resize(image, (image_dimension, image_dimension), interpolation=cv2.INTER_CUBIC)
                image = image.astype(np.float32) / 255.0  # Scale image to range [0, 1]
                images.append(image)
                processed_count += 1
            else:
                print(f"Image key not found in {file}")

            if i % 10 == 0:
                sys.stdout.write(f'\r[{i}/{len(files)}] images loaded: {i / float(len(files)) * 100:.1f} %')
                sys.stdout.flush()

        except Exception as e:
            print(f"Failed to process file {file}: {e}")

    print(f"\nFinished loading and processing data. Successfully processed {processed_count}/{len(files)} files.")
    return np.array(images)
    print(images)


# Define a function to handle the .mat files
def load_mat_file_v73(file_path):
    with h5py.File(file_path, 'r') as file:
        cjdata = file['cjdata']
        image = np.array(cjdata['image']).T  # Transpose might be necessary depending on the orientation
        tumorMask = np.array(cjdata['tumorMask']).T  # Transpose if needed
        return image, tumorMask

# Reading all .mat files
all_files = glob.glob(os.path.join(files_path, '*.mat'))
for file_path in all_files:
    image, tumorMask = load_mat_file_v73(file_path)

FCM Cluster Function

In [56]:
def tools_FCM(im, c=2, q=2):
    if not np.issubdtype(im.dtype, np.integer):
        raise ValueError("Input image must be in an integer format.")
    if np.isnan(im).any() or np.isinf(im).any():
        raise ValueError("Input image contains NaNs or Inf values.")

    # Intensity range and histogram
    Imin, Imax = np.min(im), np.max(im)
    I = np.arange(Imin, Imax + 1, dtype=np.float32)
    H, bin_edges = np.histogram(im, bins=len(I), range=(Imin, Imax))

    # Initial centroids using K-means or given centroids
    if isinstance(c, int) and c > 1:
        km = KMeans(n_clusters=c)
        weights = H.reshape(-1, 1) * I.reshape(-1, 1)
        km.fit(weights)
        C = km.cluster_centers_.flatten()
    else:
        C = np.array(c, dtype=float)

    dC = np.inf
    while dC > 1E-6:  # Convergence threshold
        C0 = C.copy()

        # Compute distances and update fuzzy memberships
        D = np.abs(I[:, np.newaxis] - C)
        D = np.power(D, 2 / (q - 1)) + np.finfo(np.float32).eps

        U = 1 / (D / np.sum(D, axis=1, keepdims=True) + np.finfo(np.float32).eps)

        # Update centroids
        UH = (U ** q) * H[:, np.newaxis]
        C = np.sum(UH * I[:, np.newaxis], axis=0) / np.sum(UH, axis=0)
        C.sort()  # Enforce natural order

        # Calculate change in centroids
        dC = np.max(np.abs(C - C0))

    # Defuzzify and create a label image
    LUT = np.argmax(U, axis=1)
    L = LUT2label(im, LUT, Imin)

    return L, C, U, LUT, H

def LUT2label(im, LUT, Imin):
    L = np.zeros_like(im, dtype=np.uint8)
    for k in np.unique(LUT):
        bw = (im == Imin + k)
        L[bw] = k + 1  # Classes are labeled starting from 1
    return L

# Testing the function
image = np.random.randint(0, 256, (100, 100), dtype=np.uint8)
L, C, U, LUT, H = tools_FCM(image, 3, 2)
print("Label Image:", L)
print("Centroids:", C)

Label Image: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Centroids: [128. 130. 141.]


Active contour function

 K-Means Function

Segmentation

In [ ]:
# Configuration for plots
plt.style.use('seaborn-v0_8-white')

# Define path and parameters
files_path = '/Users/elizabethnemeti/Desktop/data'
fuzziness = 3
numClust = 3
winSize = 7
lengthPenalty = 0.000001
iteration = 400
epsilon = 0.3

# Define a function for Fuzzy C-Means Clustering (placeholder)
def fuzzy_c_means(image, n_clusters, fuzziness):
    # Implement FCM clustering or use an existing library function
    # For actual use, you would replace this with actual FCM implementation
    # This is just a placeholder to show where FCM would be called
    pass

    # Normalize and convert image to uint8
    img = exposure.rescale_intensity(img, out_range=(0, 255)).astype(np.uint8)

    # Placeholder for Fuzzy C-Means clustering
    output_temp = fuzzy_c_means(img, numClust, fuzziness)
    img_fuzzy = (output_temp == numClust)
    img_fuzzy = morphology.remove_small_objects(img_fuzzy, min_size=5)
    img_fuzzy = morphology.binary_fill_holes(img_fuzzy)

    # Prepare for output
    output = np.zeros(img.shape, dtype=np.uint8)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    
    # Placeholder for Active Contour - use snake
    output = active_contour(img, tumorMask, alpha=winSize, beta=lengthPenalty,
                            gamma=epsilon, max_iterations=iteration)
    output = morphology.binary_fill_holes(output)

    plt.subplot(1, 2, 2)
    plt.imshow(np.dstack((img, img, img)))  # Convert grayscale to RGB
    plt.imshow(output, alpha=0.5)  # Overlay the segmentation mask
    plt.title(f'Processed Image: {os.path.basename(file_path)}')

    # Save results
    op_folder = os.path.join(os.path.dirname(file_path), 'ProcessedImages')
    os.makedirs(op_folder, exist_ok=True)
    io.imsave(os.path.join(op_folder, f'{os.path.splitext(os.path.basename(file_path))[0]}_tumor_mask.png'), output)
    masked_rgb_image = img * np.expand_dims(output, axis=-1)
    io.imsave(os.path.join(op_folder, f'{os.path.splitext(os.path.basename(file_path))[0]}_ori_masked.png'), masked_rgb_image)
    
    plt.pause(0.1)  # Short pause to update figures
    plt.close('all')  # Close plots to free memory